In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.types import LongType
from pyspark.sql import functions as F
from pyspark.sql.functions import date_format
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import isnan, when, count, col
import pandas as pd
from urllib.request import urlretrieve
import os

In [1]:

#--------------------------------------------------------------------------------------------
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)

22/09/11 20:34:51 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
22/09/11 20:34:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/11 20:34:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [48]:
exchange_rates = spark.read.csv("../data/exchange_rates.csv")
inflation = spark.read.csv("../data/inflation.csv")
merch_fees = spark.read.csv("../data/merch_fees.csv")
covid = spark.read.csv("../data/covid.csv", header=True)

In [49]:
covid.show(5)

+----------+--------------------+------------+---------+------+-----+---------+---------+----+---+----+--------+
|      date|               state|state_abbrev|confirmed|deaths|tests|positives|recovered|hosp|icu|vent|vaccines|
+----------+--------------------+------------+---------+------+-----+---------+---------+----+---+----+--------+
|2020-01-25|Australian Capita...|         ACT|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|     New South Wales|         NSW|        3|     0|    0|        3|        0|   0|  0|   0|       0|
|2020-01-25|  Northern Territory|          NT|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|          Queensland|         QLD|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|     South Australia|          SA|        0|     0|    0|        0|        0|   0|  0|   0|       0|
+----------+--------------------+------------+---------+------+-----+---------+---------+----+--

In [11]:
data = spark.read.parquet("../data/tables/full_join.parquet")

In [12]:
data.show(5)

+-------------+------------+--------------------+---------+--------------+---------------+--------------------+-----+--------+------+------------------+------------------+--------------------+--------------+-------+-----------+
|merchant_name|merchant_abn|          categories|take_rate|revenue_levels|           name|             address|state|postcode|gender|trans_merchant_abn|      dollar_value|            order_id|order_datetime|user_id|consumer_id|
+-------------+------------+--------------------+---------+--------------+---------------+--------------------+-----+--------+------+------------------+------------------+--------------------+--------------+-------+-----------+
|Felis Limited| 10023283211|furniture, home f...|     0.18|             e|      Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|       10023283211| 212.8573179553598|2430fff1-36c1-483...|    2021-05-02|   3471|        551|
|Felis Limited| 10023283211|furniture, home f...|     0.18|             e|  Adrian Mille

In [13]:
data.createOrReplaceTempView("temp")

a = spark.sql(""" 

SELECT DISTINCT(merchant_name)
FROM temp

""")

a.show()

+--------------------+
|       merchant_name|
+--------------------+
|      Amet Nulla Ltd|
|Volutpat Nulla In...|
|   Pede Praesent Ltd|
|Etiam Bibendum In...|
|Magna Ut Incorpor...|
|Faucibus Leo In C...|
|Varius Et Associates|
|Tellus Imperdiet ...|
|Molestie Tellus F...|
|Class Aptent Taci...|
|Placerat Orci Ass...|
|Vitae Purus Gravi...|
|    Dictum Magna Ltd|
|Sed Leo Cras Inco...|
|    Dui Incorporated|
|        Et Pede Inc.|
| Sagittis Associates|
|Ante Nunc Mauris ...|
|Imperdiet Ullamco...|
|     Ut Molestie LLP|
+--------------------+
only showing top 20 rows

